In [4]:
def adsrEnvelope(samples, attack = .3, decay = .25, sustain = .75, release = .02, sampleRate = 44100):
    
    #creates an adsr envelope according to function parameters
    
    import numpy as np
    
    numSamples = samples.size
    
    attackSamples = int(attack * sampleRate)
    releaseSamples = int(release * sampleRate)
    
    maxAmplitude = np.max(samples)
    
    attackArr = np.linspace(0, maxAmplitude, attackSamples)

    
    if (sustain != False):
        decaySamples = int(decay * sampleRate)
        sustainSamples = numSamples - (attackSamples + decaySamples + releaseSamples)
        if (sustainSamples < 0):
            raise Exception('Envelope is too long for duration of the given sound.')
        if (sustain > 1 or sustain < 0):
            raise Exception('Sustain should be between 0-1')
        decayArr = np.linspace(maxAmplitude, sustain, decaySamples)
        sustainArr = np.full(sustainSamples, sustain) 
        releaseArr = np.linspace(sustain, 0, releaseSamples)
        totalEnvelope = np.concatenate((attackArr, decayArr, sustainArr, releaseArr))
    else:
        releaseArr = np.linspace(maxAmplitude, 0, releaseSamples)
        leftOver = numSamples - (attackSamples + releaseSamples)
        if (leftOver < 0):
            raise Exception('Envelope is too long for duration of the given sound.')
        emptyArr = np.zeros(leftOver, dtype = int)
        totalEnvelope = np.concatenate((attackArr, releaseArr, emptyArr))
    
    return samples * totalEnvelope

In [5]:
def lowHighFilter(cutoff, sampleRate, order, signal, filterType):
    
    #applies a low or high pass filter given a cutoff, order, and filter type
    
    if cutoff < 0:
        raise Exception('Cutoff frequency cannot be negative')
    
    from scipy.signal import butter, filtfilt
    
    nyquist = sampleRate / 2
    normalCutoff = cutoff / nyquist
    (b, a) = butter(order, normalCutoff, btype = filterType)
    filteredSignal = filtfilt(b, a, signal)
    return filteredSignal

In [6]:
def genSine(frequency, duration, amplitude = 1, sampleRate = 44100, phase = 0):
    
    #creates a sine wave
    
    import numpy as np
    
    time = np.arange(0, duration, 1/sampleRate)
    return amplitude * np.sin((2*np.pi * frequency * time) + phase)

In [7]:
def genSaw(frequency, duration, numSine = 10, amplitude = 1, sampleRate = 44100, phase = 0):
    
    #creates a saw wave out of a sum of sine waves
    
    import numpy as np
    
    time = np.arange(0, duration, 1/sampleRate)
    initSine = np.sin((2*np.pi * frequency * time) + phase)
    for i in range(2, numSine + 1):
        addSine = (1/i) * np.sin((2*np.pi * (frequency * i) * time) + phase)
        initSine = initSine + addSine
    return amplitude * initSine

In [8]:
def genSquare(frequency, duration, numSine = 10, amplitude = 1, sampleRate = 44100, phase = 0):
    
    #creates a square wave out of a sum of sine waves
    
    import numpy as np
    
    time = np.arange(0, duration, 1/sampleRate)
    initSine = 0
    for i in range(1, numSine + 1, 2):
        addSine = (1/i) * np.sin((2*np.pi * (frequency * i) * time) + phase)
        initSine = initSine + addSine
    return amplitude * initSine

In [9]:
def genTriangle(frequency, duration, numSine = 10, amplitude = 1, sampleRate = 44100, phase = 0):
    
    #creates a triangle wave out of a sum of sine waves
    
    import numpy as np
    
    time = np.arange(0, duration, 1/sampleRate)
    amps = [(1/j**2) for j in range(1,numSine+1)]
    count = 1
    initSine = 0
    for i in range(1, numSine + 1):
        if (i % 2 == 0):
            addSine = 0
        else:
            addSine = (-(-1**count)/(i**2) * np.sin((2*np.pi * (frequency * i) * time) + phase))
        initSine = initSine + addSine
        count = count + 1
    return amplitude * initSine

In [10]:
def addDelay(samples, sampleRate, delayTime, delayFactor):
    
    # creates a new signal with delay added where for a given delay factor n, there are n repeated
    # delays that decrease in amplitude by a factor of 1/n^2
    
    import numpy as np
    
    pad = np.zeros(int((sampleRate/1000)*delayTime))
    result = np.zeros(samples.size + delayFactor*pad.size)
    for i in range (0, delayFactor):
        front = np.zeros(int((sampleRate/1000)*delayTime*i))
        end = np.zeros(int((sampleRate/1000)*delayTime*(delayFactor - i)))
        adjustAmp = (1/(i*i+1))*samples
        add = np.concatenate([front, adjustAmp, end])
        result += add;
    return result

In [11]:
def amplitudeModulation(samples, duration, frequency, waveform):
    
    #performs amplitude modulation on a given signal 
    
    if waveform == 'Sine':
        amp = genSine(frequency, duration)
    elif waveform == 'Saw':
        amp = genSaw(frequency, duration, 20)
    elif waveform == 'Square':
        amp = genSquare(frequency, duration, 20)
    elif waveform == 'Triangle':
        amp = genTriangle(frequency, duration, 20)
    return samples*amp